# Rabi Cycling in the Two-Level-System — Lab and Rotating Frame

In [1]:
from sympy import Function, symbols, sqrt, Eq, Abs, exp, cos, sin, Matrix, dsolve, solve

In [2]:
from sympy import I as 𝕚

In [3]:
g = Function('g')
e = Function('e')
t = symbols('t', positive=True)
Δ = 0#symbols('Delta', real=True)
Ω0 = symbols('Ω_0', real=True)
Ω = symbols('Omega', real=True)

In [4]:
Ĥ = Matrix([
    [  0  , -Ω0/2],
    [-Ω0/2,    Δ ]
])
Ĥ

Matrix([
[     0, -Ω_0/2],
[-Ω_0/2,      0]])

In [5]:
TDSE_system = [
    g(t).diff(t) + 𝕚 * (Ĥ[0,0] * g(t) + Ĥ[0,1] * e(t)),
    e(t).diff(t) + 𝕚 * (Ĥ[1,0]*  g(t) + Ĥ[1,1] * e(t)),
]

In [6]:
sols_gen = dsolve(TDSE_system, [g(t), e(t)])

Note that `dsolve` allows to specify boundary conditions, but the resulting expressions are hard to simply. We do better by solving for the integration constants "manually" later on.

In [7]:
effective_rabi_freq = {
    Ω: sqrt(Δ**2 + Ω0**2)
}

In [8]:
Eq(Ω, effective_rabi_freq[Ω])

Eq(Omega, Abs(Ω_0))

In [9]:
find_Ω = {
    effective_rabi_freq[Ω]: Ω
}

In [10]:
sols_gen[0].subs(find_Ω)

Eq(g(t), -C1*exp(-I*t*Ω_0/2) + C2*exp(I*t*Ω_0/2))

In [11]:
sols_gen[1].subs(find_Ω).expand()

Eq(e(t), C1*exp(-I*t*Ω_0/2) + C2*exp(I*t*Ω_0/2))

In [12]:
boundary_conditions = {
    t: 0,
    g(t): 1,
    e(t) : 0,
}

In [13]:
find_Ω0sq = {
    Ω**2 - Δ**2: Ω0**2
}

In [14]:
C1, C2 = symbols("C1, C2")

In [15]:
_integration_constants = solve(
    [sol.subs(find_Ω).subs(boundary_conditions) for sol in sols_gen],
    [C1, C2]
)
integration_constants = {
    k: v.subs(find_Ω0sq)
    for (k, v) in _integration_constants.items()
}

In [16]:
Eq(C1, integration_constants[C1])

Eq(C1, -1/2)

In [17]:
Eq(C2, integration_constants[C2])

Eq(C2, 1/2)

In [18]:
global_phase = exp(-𝕚 * Δ * t / 2)

In [19]:
def remove_global_phase(eq, global_phase=global_phase):
    return Eq(eq.lhs, eq.rhs * global_phase.conjugate())

def restore_global_phase(eq, global_phase=global_phase):
    return Eq(eq.lhs, eq.rhs * global_phase)

In [20]:
def collect(eq, term):
    return Eq(eq.lhs, eq.rhs.collect(term))

In [21]:
def simplify_sol_g(sol_g):
    rhs = (
        sol_g
        .rhs
        .rewrite(sin)
        .expand()
        .collect(𝕚)
        .subs(effective_rabi_freq)
        .simplify()
        .subs(find_Ω)
    )
    return Eq(sol_g.lhs, rhs)

In [22]:
sol_g = restore_global_phase(
    simplify_sol_g(
        collect(
            remove_global_phase(
                sols_gen[0]
                .subs(find_Ω)
                .subs(integration_constants)
            ).expand(),
            exp(𝕚 * Ω * t / 2                                                                   )
        )
    )
)
sol_g

Eq(g(t), cos(t*Ω_0/2))

In [23]:
sol_e = restore_global_phase(
    remove_global_phase(
        sols_gen[1].subs(find_Ω).subs(integration_constants)
    ).expand().rewrite(sin).expand()
)
sol_e

Eq(e(t), I*sin(t*Ω_0/2))

In [24]:
pop_e = (sol_e.rhs * sol_e.rhs.conjugate())
Eq(Abs(e(t))**2, pop_e)

Eq(Abs(e(t))**2, sin(t*Ω_0/2)**2)

In [25]:
Ψ = Matrix([sol_g.rhs, sol_e.rhs])
Ψ

Matrix([
[  cos(t*Ω_0/2)],
[I*sin(t*Ω_0/2)]])

In [26]:
ωₗ = symbols("omega_l", positive=True)

In [27]:
U = Matrix([[1, 0], [0, exp(-𝕚*ωₗ*t)]])
U

Matrix([
[1,                 0],
[0, exp(-I*omega_l*t)]])

In [28]:
U_dag = U.conjugate().transpose()
U_dag

Matrix([
[1,                0],
[0, exp(I*omega_l*t)]])

In [29]:
Ψ_lab = U_dag @ Ψ
Ψ_lab

Matrix([
[                   cos(t*Ω_0/2)],
[I*exp(I*omega_l*t)*sin(t*Ω_0/2)]])

In [30]:
σx = Matrix([[0, 1], [1, 0]])
σx

Matrix([
[0, 1],
[1, 0]])

In [31]:
σy = Matrix([[0, -𝕚], [𝕚, 0]])
σy

Matrix([
[0, -I],
[I,  0]])

In [32]:
def inner(a, b):
    return a.dot(b, hermitian=True, conjugate_convention="left")

In [33]:
def expval(O, Ψ):
    return inner(Ψ, O @ Ψ).simplify()

In [34]:
expval(σx, Ψ)

0

In [35]:
expval(σx, Ψ_lab)

-sin(omega_l*t)*sin(t*Ω_0)

In [36]:
expval(σy, Ψ)

sin(t*Ω_0)

In [37]:
expval(σy, Ψ_lab)

sin(t*Ω_0)*cos(omega_l*t)